In [ ]:
# Install cell, Install if needed

#pip install sklearn
#pip install Keras-Preprocessing

In [51]:
#import block

import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from typing import Union
import matplotlib.pyplot as plt
from keras_flops import get_flops
import pandas as pd
#ResNet Resource
#https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
import tensorflow.keras as K

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from keras_preprocessing.image import ImageDataGenerator
import numpy as np
import os
from PIL import Image
from skimage import io

tf.enable_v2_behavior()

# Load Data

In [63]:
#load data use panda

#read annotation file
annotations = pd.read_csv("./annotations.csv", skiprows = 1,
                         names = ["Image Name","Majority Vote Label","Number of Annotations who Selected SSA(Out of 7)","Partition"])
annotations.head()

#Create dataframe
df = pd.DataFrame(annotations,
                 columns = ["Image Name","Majority Vote Label","Number of Annotations who Selected SSA(Out of 7)", "Partition"])
df.set_index("Image Name",inplace = True)

df.head()

,Majority Vote Label,Number of Annotations who Selected SSA(Out of 7),Partition
Image Name,,,
MHIST_aaa.png,SSA,6,train
MHIST_aab.png,HP,0,train
MHIST_aac.png,SSA,5,train
MHIST_aae.png,HP,1,train
MHIST_aaf.png,SSA,5,train


# Data Augmentation

In [62]:
#resource: https://www.youtube.com/watch?v=ccdssX4rIh8



def create_images(partition,label):
    SIZE = 224
    image_directory = "images/"
    data_set = []
    
    datagen = ImageDataGenerator(
        rotation_range = 180,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.5,
        horizontal_flip = True,
        fill_mode = 'nearest'
    )
    
    my_images = os.listdir(image_directory)
    length = 0
    for i,image_name in enumerate(my_images):
        if df.loc[image_name][2] == partition and df.loc[image_name][0] == label:
            image = io.imread(image_directory+image_name)
            image = Image.fromarray(image,'RGB')
            image = image.resize((SIZE,SIZE))
            data_set.append(np.array(image))
            length = length + 1
    x = np.array(data_set)
    i = 0
    for batch in datagen.flow(x,
                              batch_size = 1,
                              save_to_dir = partition,
                              save_prefix = label,
                              save_format = 'png'):
        i = i + 1
        #change length to change the number of images generates
        if i < length:
            continue
        else:
            break

#create directory before run the code
create_images('train','SSA')
create_images('test','SSA')
create_images('train','HP')
create_images('test','HP')

630
360


# Create Model